In [ ]:
import pandas as pd
import numpy as np
import xml.etree.cElementTree as ET

In [ ]:
filepath = r'network/random-routes-file_2024-02-24_md4500_ff50.rou.xml' # path to random trips file

In [ ]:
tree = ET.parse(filepath)

In [ ]:
root = tree.getroot()

In [ ]:
ids = []
depart = []
edges = []

count = 0

for vehicle in root:
    for route in vehicle:
        ids.append(vehicle.attrib['id'])
        depart.append(vehicle.attrib['depart'])
        edges.append(np.array(route.attrib['edges'].split(' ')))
    count += 1
    
df = pd.DataFrame({'id':ids, 'depart':depart, 'edges':edges})

In [ ]:
df

## Inspect

#### Load SUMO road network

In [ ]:
## get sumo network
import sys
sys.path.append(r'/mnt/c/users/ty90rize/Sumo/tools') # path to sumolib
import sumolib

In [ ]:
path_osm = 'network/osm_final_2024-02-25.net.xml'

In [ ]:
net_osm = sumolib.net.readNet(path_osm)

## Detect u-turns and remove

In [ ]:
def detect_uturn(net, route):
    prior_previous_id = None
    prior_next_id = None
    prior_id = None
    for edge_id in route:
        edge = net.getEdge(edge_id)
        previous_id = edge.getFromNode().getID()
        next_id = edge.getToNode().getID()
        if prior_id is not None:
            if (previous_id == prior_next_id) and (next_id == prior_previous_id):
                return 1
        prior_id = edge_id
        prior_next_id = next_id
        prior_previous_id = previous_id
    return 0

In [ ]:
remove = df['edges'].apply(lambda x: detect_uturn(net_osm, x))
df['remove'] = remove

In [ ]:
print('{} routes are going to be removed.'.format(sum(remove)))

## Copied manually from source file

In [ ]:
attrib = {'xmlns:xsi':"http://www.w3.org/2001/XMLSchema-instance", 
          'xsi:noNamespaceSchemaLocation':"http://sumo.dlr.de/xsd/routes_file.xsd"}

In [ ]:
new_root = ET.Element(root.tag)

In [ ]:
new_root.attrib = attrib

## Create new df

In [ ]:
from datetime import datetime
now = datetime.now()
timesuffix = now.strftime("%y%m%d_%H%M%S")

In [ ]:
for idx, row in df.iterrows():
    if row['remove'] == 0:
        vehicle = ET.SubElement(new_root, "vehicle", id=row['id'], depart = row['depart'])
        route = ET.SubElement(vehicle, "route", edges = " ".join(row['edges']))

new_tree = ET.ElementTree(new_root)
ET.indent(new_tree, space = "\t", level = 0)
new_tree.write('network/{}_{}.rou.xml'.format(filepath[:-8], timesuffix))